# Job Execution

## Authentication -- optional step
If you have registered a client (see the authentidation use case), uncomment the code below to generate an access token.

In [ ]:
# import requests, json, base64
# sasserver = ""
# username = ""
# password = ""
# client_id = ""
# client_secret = ""
# url = sasserver + "/SASLogon/oauth/token"
# data = {
#     'grant_type': 'password',
#     'username': username,
#     'password': password
# }
# headers = {'Accept': 'application/json'}
# response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret), verify=False).json()
# access_token = response["access_token"]
# print("The access token is: " + access_token)

## Import modules, variable assignment
The first step of the process is to import the required packages and assign variable values.

In [ ]:
import requests
import json

# Variables to assign
sasserver = ""
folder = "Public"
access_token = ""

## Get Public Folder (or directory of your choice)

In [ ]:
url = sasserver+"/folders/folders?filter=eq(name,"+folder+")"

payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()

folder_id = response["items"][0]["id"]

print(folder_id)

## Create Folder

In [ ]:
url = sasserver+"/folders/folders?parentFolderUri=/folders/folders/"+folder_id

payload="{\n  \"name\": \"API Job Exec Python\",\n  \"description\": \"My API tests folder.\",\n  \"type\": \"folder\"\n}"
headers = {
  'Content-Type': 'application/vnd.sas.content.folder+json',
  'Accept': 'application/vnd.sas.content.folder+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False).json()

parentFolder = response["id"]

print(parentFolder)

## Create Job Definition

In [ ]:
url = sasserver+"/jobDefinitions/definitions?parentFolderUri=/folders/folders/"+parentFolder

payload="{\n\"version\":2,\n\"name\":\"Simple proc print\",\n\"description\": \
\"Show the contents of sashelp.class using PROC PRINT\",\n\"type\":\"Compute\", \
\n\"parameters\":[\n{\n\"version\": 1,\n\"name\":\"_contextName\",\n\"defaultValue\": \
\"SAS Job Execution compute context\",\n\"type\":\"CHARACTER\",\n\"label\":\"Context Name\",\n \
\"required\": false\n}\n],\n\"code\":\"ods html style=HTMLBlue;\\nproc print data=sashelp.class; run; quit;\\nods \
html close;\"\n}"
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False).json()

jobDef = response["id"]

print(jobDef)

## Get Job Definition - just verifying Job was created

In [ ]:
url = sasserver+"/jobDefinitions/definitions/"+jobDef

payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()

defId = response["id"]

print(defId)

## Submit Job Execution

In [ ]:
url = sasserver+"/jobExecution/jobs"

payload={"name": "Hello World Execution",
         "description": "Execution of the job we previously created",
         "jobDefinitionUri": "/jobDefinitions/definitions/" + defId
        }

headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload), verify=False).json()

execution_id = response["id"]

print(execution_id)

## Get Job Execution State

In [ ]:
# Re-run this step until "completed" state is returned

url = sasserver+"/jobExecution/jobs/"+execution_id+"/state"

payload={}

headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

## Get Job Execution Files

In [ ]:
# Do not run this step until state = "completed" is returned in the previous call

url = sasserver+"/jobExecution/jobs/"+execution_id

payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()

print(response)

files_location = response["results"]["sashtml.htm"]
# folder_id = response["items"][0]["id"]


print("HTML file location: " + files_location)

## Get Execution Results

In [ ]:
url = sasserver+files_location+"/content"

payload={}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)